## ANALISIS TRACK BASADO EN PWD

In [1]:
#PASO_1
#Importamos un xml procedente de pwd

from xml.dom.minidom import parse
import xml.dom.minidom
import pandas as pd

path='Mallorca70.3_tp'
DOMTree = xml.dom.minidom.parse(path+".xml")


collection = DOMTree.documentElement

trkpts = collection.getElementsByTagName("sample")
list_track = []

count=0
for trkpt in trkpts:
    count +=1
    if len(trkpt.getElementsByTagName("timeoffset"))==0:
        timeoffset = ''
    else:
        timeoffset = float(trkpt.getElementsByTagName("timeoffset")[0].firstChild.nodeValue)
    
    if len(trkpt.getElementsByTagName("hr"))==0:
        hr = ''
    else:
        hr = float(trkpt.getElementsByTagName("hr")[0].firstChild.nodeValue)
    
    if len(trkpt.getElementsByTagName("spd"))==0:
        spd = ''
    else:
        spd = float(trkpt.getElementsByTagName("spd")[0].firstChild.nodeValue)    
    
    if len(trkpt.getElementsByTagName("pwr"))==0:
        pwr = ''
    else:
        pwr = float(trkpt.getElementsByTagName("pwr")[0].firstChild.nodeValue)

    if len(trkpt.getElementsByTagName("cad"))==0:
        cad = ''
    else:
        cad = float(trkpt.getElementsByTagName("cad")[0].firstChild.nodeValue)

    if len(trkpt.getElementsByTagName("dist"))==0:
        dist = ''
    else:
        dist = float(trkpt.getElementsByTagName("dist")[0].firstChild.nodeValue)

    if len(trkpt.getElementsByTagName("lat"))==0:
        lat = ''
    else:
        lat = float(trkpt.getElementsByTagName("lat")[0].firstChild.nodeValue)

    if len(trkpt.getElementsByTagName("lon"))==0:
        lon = ''
    else:
        lon = float(trkpt.getElementsByTagName("lon")[0].firstChild.nodeValue)

    if len(trkpt.getElementsByTagName("alt"))==0:
        alt = ''
    else:
        alt = float(trkpt.getElementsByTagName("alt")[0].firstChild.nodeValue)    

    if len(trkpt.getElementsByTagName("temp"))==0:
        temp = ''
    else:
        temp = float(trkpt.getElementsByTagName("temp")[0].firstChild.nodeValue)
   
    list_track.append([timeoffset, hr, spd, pwr, cad, dist, lat, lon, alt, temp])

print ("Puntos cargados: %d" %count)

    

result = pd.DataFrame(list_track)
result.columns=["timeoffset", "hr", "spd", "pwr", "cad", "dist", "lat", "lon", "alt", "temp"]
result['prueba'] = path
result.to_excel('Intermedio'+path+'.xlsx')
result.to_excel(path+".xlsx")
print('finish')



Puntos cargados: 0


ValueError: Length mismatch: Expected axis has 0 elements, new values have 10 elements

## ANALISIS TRACK BASADO EN GPX

In [5]:
#PASO_1
#Importamos un xml procedente de gpx

from xml.dom.minidom import parse
import xml.dom.minidom
import pandas as pd


#path='Analisis_Aerodinamica'
#path='Entreno_preparatorio'
#path='2017_05_13_Triatlón'
path='20180406_LEVS_195_90_R_E'
DOMTree = xml.dom.minidom.parse(path+".xml")

collection = DOMTree.documentElement

trkpts = collection.getElementsByTagName("trkpt")
list_track = []

def to_dt(dato):
    t=pd.to_datetime(dato)
    return t.hour*3600+t.minute*60+t.second

count=0

for trkpt in trkpts:
    count +=1
    if trkpt.hasAttribute("lat"):
      lat = float(trkpt.getAttribute("lat"))
    if trkpt.hasAttribute("lon"):
      lon = float(trkpt.getAttribute("lon"))
    
    if len(trkpt.getElementsByTagName("ele"))==0:
        alt = ''
    else:
        alt = float(trkpt.getElementsByTagName("ele")[0].firstChild.nodeValue)
    
    if len(trkpt.getElementsByTagName("time"))==0:
        time = ''
    else:
        time = trkpt.getElementsByTagName("time")[0].firstChild.nodeValue
    
    if len(trkpt.getElementsByTagName("gpxdata:hr"))==0:
        hr = ''
    else:
        hr = float(trkpt.getElementsByTagName("gpxdata:hr")[0].firstChild.nodeValue)
    
    if len(trkpt.getElementsByTagName("gpxdata:cadence"))==0:
        cad = ''
    else:
        cad = float(trkpt.getElementsByTagName("gpxdata:cadence")[0].firstChild.nodeValue)
        
    if len(trkpt.getElementsByTagName("gpxdata:temp"))==0:
        temp = ''
    else:
        temp = float(trkpt.getElementsByTagName("gpxdata:temp")[0].firstChild.nodeValue)
    
    if len(trkpt.getElementsByTagName("gpxdata:distance"))==0:
        dist = ''
    else:
        dist = float(trkpt.getElementsByTagName("gpxdata:distance")[0].firstChild.nodeValue)
    
    if len(trkpt.getElementsByTagName("gpxdata:power"))==0:
        pwr = ''
    else:
        pwr = float(trkpt.getElementsByTagName("gpxdata:power")[0].firstChild.nodeValue)

    if len(trkpt.getElementsByTagName("gpxdata:speed"))==0:
        spd = ''
    else:
        spd = float(trkpt.getElementsByTagName("gpxdata:speed")[0].firstChild.nodeValue)
    
    timeoffset = to_dt(time)
    
    list_track.append([timeoffset, hr, spd, pwr, cad, dist, lat, lon, alt, temp])

result = pd.DataFrame(list_track)
result.columns = ["timeoffset", "hr", "spd", "pwr", "cad", "dist", "lat", "lon", "alt", "temp"]
result['prueba'] = path
print ("Puntos cargados: %d" %count)
result.to_excel('Intermedio'+path+'.xlsx')
print('finish')


Puntos cargados: 7586
finish


In [ ]:
#conda install -c conda-forge geopy

## ENRIQUECEMOS EL FICHERO. COMUN PARA TODOS LOS ORIGENES

In [7]:
import pandas as pd
import math as mt
from scipy.signal import savgol_filter

#path='Analisis_Aerodinamica'
#path='Entreno_preparatorio'
#path='2017_05_13_Triatlón'
path='20180406_LEVS_195_90_R_E'

df = pd.read_excel('Intermedio'+path+'.xlsx')

df2 = df[df['lat'] >0]
df2['pwr'].fillna(0, inplace=True)
#df2.drop(['dist','cad','hr','spd'], axis=1, inplace=True)

from geopy.distance import distance
from geopy import Point
i=1
dist_c=[]
dist_c.append(0)
ele_gain = []
ele_gain.append(0)
d=0
e=0
porc=[]
porc.append(0)
spd_c=[]
spd_c.append(0)
spd_prev=[]
spd_prev.append(0)
ele_prev=[]
ele_prev.append(0)
bearing=[]
bearing.append(0)
timeoffset_prev=[]
timeoffset_prev.append(0)


while i < len(df2):
    #print("Punto %d" %i
    punto1 = Point(df2.iloc[i-1]['lon'], df2.iloc[i-1]['lat'])
    punto2 = Point(df2.iloc[i]['lon'], df2.iloc[i]['lat'])
    dist_c.append(distance(punto1, punto2).m)
    d = distance(punto1, punto2).m
    spd_c.append(float(d/(df2.iloc[i]['timeoffset']-df2.iloc[i-1]['timeoffset'])))
    punto1 = df2.iloc[i-1]['alt']
    punto2 = df2.iloc[i]['alt']
    ele_gain.append((punto2)-(punto1))
    e = punto2-punto1
    porc.append(e*100/d)
    lona = df2.iloc[i-1]['lon']
    lata = df2.iloc[i-1]['lat']
    lonb = df2.iloc[i]['lon']
    latb = df2.iloc[i]['lat']
    X = mt.cos(lonb)*mt.sin(latb-lata)
    Y = mt.cos(lona)*mt.sin(lonb)-mt.sin(lona)*mt.cos(lonb)*mt.cos(latb-lata)
    bearing.append(mt.atan2(X,Y)*57.2957795)
    i+=1
df2['dist_c']=dist_c
df2['ele_gain']=ele_gain
df2['porc']=porc
df2['spd_c']=spd_c
df2['bearing']=bearing
i=1
while i < len(df2):
    spd_prev.append(df2.iloc[i-1]['spd_c'])
    ele_prev.append(df2.iloc[i-1]['alt'])
    timeoffset_prev.append(df2.iloc[i-1]['timeoffset'])
    i+=1
df2['spd_prev']=spd_prev
df2['ele_prev']=ele_prev
df2['timeoffset_prev']=timeoffset_prev

#aplicamos el suavizado de señal
axis_y = df["alt"]
axis_yhat = savgol_filter(axis_y, 31, 2)
df2["new_ele"] = axis_yhat

tramos = [0]
tramo = 0
pendiente = 0
pendiente_ant = 0

i = 1
print("Entramos en el While1 %d" % len(df2.index))
while i < len(df2.index):
    #print("punto = %d y tramo = %d" % (i, tramo))
    pendiente = df2.iloc[i]["new_ele"]-df2.iloc[i-1]["new_ele"]

    if pendiente * pendiente_ant >=0:
        tramos.append(tramo)
    else:
        tramo +=1
        tramos.append(tramo)
    i+=1
    pendiente_ant = pendiente

print("tramos : %d" % len(tramos))
print("Salimos del While")
df2["tramo"] = tramos

#Calculo de tramos de 100m

tramos_100 = [0]
tramo = 0
acum_dist = 0
i = 1

print("Entramos en el While2 %d" % len(df2.index))

while i < len(df2.index):
    acum_dist = df2.iloc[i-1]["dist"]
    tramo = mt.floor(acum_dist/100)
    tramos_100.append(tramo)
    i +=1
print("tramos_100m : %d" % len(tramos_100))
print("Salimos del While2")
df2["tramos_100"] = tramos_100






df2.to_excel(path+'_ml.xlsx')

print('finish')


C:\Users\David\Anaconda3\lib\site-packages\ipykernel_launcher.py:45: RuntimeWarning: divide by zero encountered in true_divide


Entramos en el While1 7586
tramos : 7586
Salimos del While
Entramos en el While2 7586
tramos_100m : 7586
Salimos del While2
finish


In [8]:
df2

,timeoffset,hr,spd,pwr,cad,dist,lat,lon,alt,temp,...,ele_gain,porc,spd_c,bearing,spd_prev,ele_prev,timeoffset_prev,new_ele,tramo,tramos_100
0,27865,NaN,5.000000,0.000000,0,0.000000,40.519800,-3.929805,662.799988,22.700001,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,661.131985,0,0
1,27867,NaN,5.141748,0.000000,0,15.000000,40.519850,-3.929899,661.417483,22.700001,...,-1.382505,-11.731324,5.892366,-159.442814,0.000000,662.799988,27865,660.841092,0,0
2,27868,NaN,5.200000,0.000000,0,26.000000,40.519883,-3.930019,661.610010,22.600000,...,0.192527,1.398533,13.766347,-169.028497,5.892366,661.417483,27867,660.532461,0,0
3,27870,NaN,5.238999,0.000000,0,39.000000,40.519872,-3.930170,659.610010,22.600000,...,-2.000000,-11.945909,8.371067,177.060612,13.766347,661.610010,27868,660.206093,0,0
4,27872,NaN,5.300000,0.000000,0,52.000000,40.519832,-3.930314,659.000000,22.600000,...,-0.610010,-3.689984,8.265749,168.923616,8.371067,659.610010,27870,659.861987,0,0
5,27874,NaN,5.352179,0.000000,0,65.000000,40.519736,-3.930410,658.869553,22.600000,...,-0.130447,-0.867023,7.522692,144.828871,8.265749,659.000000,27872,659.500143,0,0
6,27876,NaN,6.059540,0.000000,0,81.000000,40.519628,-3.930525,658.000000,22.600000,...,-0.869553,-4.974327,8.740409,146.507487,7.522692,658.869553,27874,659.120562,0,0
7,27878,NaN,6.338999,0.000000,0,93.000000,40.519561,-3.930643,658.000000,22.600000,...,0.000000,0.000000,7.510466,158.197601,8.740409,658.000000,27876,658.723243,0,0
8,27880,NaN,6.400000,0.000000,0,106.000000,40.519532,-3.930786,658.000000,22.600000,...,0.000000,0.000000,8.068761,171.870085,7.510466,658.000000,27878,658.308186,0,0
9,27882,NaN,6.400000,0.000000,0,124.706971,40.519534,-3.931000,657.000000,22.600000,...,-1.000000,-4.225640,11.832527,-179.622889,8.068761,658.000000,27880,657.875392,0,1
